In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import csv
import json
from lxml.html import parse
from urllib.request import urlopen

### python有许多可以读写HTML和xml格式数据的库.lxml就是其中之一,它能高效且可靠的解析大文件.lxml多个编程接口.首先需要用lxml.html处理HTML,再用lxml.objectify做一些XML处理

In [2]:
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()

### 通过这个对象,可以获取特定类型的所有HTML标签,比如含有所需数据的table标签.

In [3]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x1d7e7b2f9a8>,
 <Element a at 0x1d7e7b2f9f8>,
 <Element a at 0x1d7e7b2fa48>,
 <Element a at 0x1d7e7b2fa98>,
 <Element a at 0x1d7e7b2fae8>]

### 以上获取的是表示HTML元素的对象.要得到URL和链接文本,必须使用各对象的get方法(针对URL)和text_content方法(针对显示文本)

In [4]:
lnk = links[28]
lnk

<Element a at 0x1d7e7b2fdb8>

In [5]:
lnk.get('href')

'/quote/AAPL/options?strike=false&straddle=false'

In [6]:
lnk.text_content()

'40.00'

### 通过列表推导式获取全部的URL

In [7]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['/',
 '/watchlists',
 '/portfolios?bypass=true',
 '/screener',
 '/calendar',
 '/industries',
 '/personal-finance',
 '/tech',
 '/topic/yahoo-finance-podcast',
 '/live/facebook']

### 从文档中找出正确表格的办法就是反复实验.有些网站会给目标表格加上一个ID属性

In [9]:
tables = doc.findall('.//table')
tables

[<Element table at 0x1d7e7b2f368>, <Element table at 0x1d7e7b40188>]

In [10]:
tables[0]
tables[1]

<Element table at 0x1d7e7b40188>

### 每个表格都一个标题行,然后才是数据行

In [11]:
rows = tables[0].findall('.//tr')
rows

[<Element tr at 0x1d7e50c29f8>,
 <Element tr at 0x1d7e50c29a8>,
 <Element tr at 0x1d7e50c27c8>,
 <Element tr at 0x1d7e50c2908>,
 <Element tr at 0x1d7e50c2278>,
 <Element tr at 0x1d7e50c2c78>,
 <Element tr at 0x1d7e50c2cc8>,
 <Element tr at 0x1d7e50c2d18>,
 <Element tr at 0x1d7e50c2d68>,
 <Element tr at 0x1d7e50c2db8>,
 <Element tr at 0x1d7e50c2e08>,
 <Element tr at 0x1d7e50c2e58>,
 <Element tr at 0x1d7e50c2ea8>,
 <Element tr at 0x1d7e50c2ef8>,
 <Element tr at 0x1d7e50c2f48>,
 <Element tr at 0x1d7e50c2f98>,
 <Element tr at 0x1d7e50c3048>,
 <Element tr at 0x1d7e50c3098>,
 <Element tr at 0x1d7e50c30e8>,
 <Element tr at 0x1d7e50c3138>,
 <Element tr at 0x1d7e50c3188>,
 <Element tr at 0x1d7e50c31d8>,
 <Element tr at 0x1d7e50c3228>,
 <Element tr at 0x1d7e50c3278>,
 <Element tr at 0x1d7e50c32c8>,
 <Element tr at 0x1d7e50c3318>,
 <Element tr at 0x1d7e50c3368>,
 <Element tr at 0x1d7e50c33b8>,
 <Element tr at 0x1d7e50c3408>,
 <Element tr at 0x1d7e50c3458>,
 <Element tr at 0x1d7e50c34a8>,
 <Elemen

### 对于标题行和数据行,需要获取每个单元格内的文本.对于标题行,就是th单元格,而对于数据行,则是td单元格

In [12]:
def _unpack(row,kind='td'):
    elts = row.findall('.//%s'%kind)
    return [val.text_content() for val in elts]

In [13]:
_unpack(rows[0],kind='th')

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

### 将数据转换为一个DF,由于数值型数据任然是字符串格式,所以将部分列转换为浮点数格式.可以使用pandas.textparser进行自动类型转换

In [15]:
from pandas.io.parsers import TextParser
def parse_option_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0],kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data,names=header).get_chunk()

### 对lxml表格对象调用解析函数并得到最终的DF

In [16]:
call_data = parse_option_data(tables[0])
put_data = parse_option_data(tables[1])
call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL180119C00002500,2017-12-27 12:39PM EST,2.5,168.04,166.15,167.40,0.00,-,6,3,0.00%
1,AAPL180119C00005000,2017-11-09 2:55PM EST,5.0,170.37,169.30,170.10,0.00,-,99,0,0.00%
2,AAPL180119C00010000,2017-11-16 11:06AM EST,10.0,161.00,159.00,161.00,0.00,-,5,5,0.00%
3,AAPL180119C00017500,2017-11-28 11:15AM EST,17.5,155.83,151.50,152.45,22.98,+17.30%,1,1,0.00%
4,AAPL180119C00025000,2017-11-09 3:29PM EST,25.0,150.40,149.35,150.15,0.00,-,"2,000",0,0.00%
5,AAPL180119C00035000,2017-11-09 3:29PM EST,35.0,140.50,139.40,140.20,0.00,-,"2,000",0,0.00%
6,AAPL180119C00040000,2017-11-17 11:14AM EST,40.0,130.72,129.10,131.10,-4.73,-3.49%,4,0,0.00%
7,AAPL180119C00042500,2017-12-19 3:40PM EST,42.5,132.10,131.80,133.25,0.00,-,100,201,0.00%
8,AAPL180119C00047500,2018-01-02 3:28PM EST,47.5,124.41,127.20,127.85,0.00,-,10,17,0.00%
9,AAPL180119C00050000,2018-01-12 1:23PM EST,50.0,126.80,126.75,127.30,1.65,+1.32%,70,720,478.13%
